In [40]:
import numpy as np
from utils import get_data, check_results
from iou import calculate_ious

In [2]:
def calculate_iou(gt_bbox, pred_bbox):
    """
    calculate iou 
    args:
    - gt_bbox [array]: 1x4 single gt bbox
    - pred_bbox [array]: 1x4 single pred bbox
    returns:
    - iou [float]: iou between 2 bboxes
    """
    
    # intersect box cordinates
    xmin = np.max([gt_bbox[0], pred_bbox[0]])
    ymin = np.max([gt_bbox[1], pred_bbox[1]])
    xmax = np.min([gt_bbox[2], pred_bbox[2]])
    ymax = np.min([gt_bbox[3], pred_bbox[3]])
    
    intersection = max(0, xmax - xmin) * max(0, ymax - ymin) # area
    gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
    pred_area = (pred_bbox[2] - pred_bbox[0]) * (pred_bbox[3] - pred_bbox[1])
    
    union = gt_area + pred_area - intersection
    return intersection / union


def calculate_ious(gt_bboxes, pred_bboxes):
    """
    calculate ious between 2 sets of bboxes 
    args:
    - gt_bboxes [array]: Nx4 ground truth array
    - pred_bboxes [array]: Mx4 pred array
    returns:
    - iou [array]: NxM array of ious
    """
    ious = np.zeros((gt_bboxes.shape[0], pred_bboxes.shape[0]))
    
    for i, gt_bbox in enumerate(gt_bboxes):
        for j, pred_bbox in enumerate(pred_bboxes):
            ious[i,j] = calculate_iou(gt_bbox, pred_bbox)
    return ious

In [3]:
ground_truth, predictions = get_data()

In [19]:
predictions[0]

{'boxes': [[783, 1104, 1011, 1700],
  [853, 0, 1220, 200],
  [734, 0, 1100, 240],
  [753, 474, 868, 609],
  [830, 1500, 1004, 1914]],
 'classes': [1, 2, 1, 2, 1],
 'filename': 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'}

In [29]:
# get bboxes array
filename = 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'

In [30]:
gt_bboxes = [g['boxes'] for g in ground_truth if g['filename'] == filename][0]
gt_bboxes = np.array(gt_bboxes)

In [31]:
pred_bboxes = [p['boxes'] for p in predictions if p['filename'] == filename][0]
pred_boxes = np.array(pred_bboxes)

In [32]:
gt_bboxes

array([[ 793, 1134, 1001, 1718],
       [ 737,    0,  898,  260],
       [ 763,  484,  878,  619],
       [ 734,    0, 1114,  277],
       [ 853,    0, 1280,  250],
       [ 820, 1566,  974, 1914],
       [ 762,  951,  844, 1175],
       [ 748,  197,  803,  363]])

In [33]:
pred_boxes

array([[ 783, 1104, 1011, 1700],
       [ 853,    0, 1220,  200],
       [ 734,    0, 1100,  240],
       [ 753,  474,  868,  609],
       [ 830, 1500, 1004, 1914]])

In [35]:
gt_bboxes.shape

(8, 4)

In [36]:
pred_boxes.shape

(5, 4)

In [38]:
ious = calculate_ious(gt_bboxes, pred_boxes)
ious

array([[0.84313051, 0.        , 0.        , 0.        , 0.23860974],
       [0.        , 0.08469791, 0.4243356 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.73221757, 0.        ],
       [0.        , 0.41277874, 0.83450504, 0.        , 0.        ],
       [0.        , 0.68758782, 0.43810509, 0.        , 0.        ],
       [0.12221933, 0.        , 0.        , 0.        , 0.66359447],
       [0.02888778, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.02499868, 0.        , 0.        ]])

In [39]:
check_results(ious)

Congrats, the iou calculation is correct!


In [41]:
ious

array([[0.84313051, 0.        , 0.        , 0.        , 0.23860974],
       [0.        , 0.08469791, 0.4243356 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.73221757, 0.        ],
       [0.        , 0.41277874, 0.83450504, 0.        , 0.        ],
       [0.        , 0.68758782, 0.43810509, 0.        , 0.        ],
       [0.12221933, 0.        , 0.        , 0.        , 0.66359447],
       [0.02888778, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.02499868, 0.        , 0.        ]])

In [46]:
xs, ys = np.where(ious>0.5)

In [47]:
xs

array([0, 2, 3, 4, 5])

In [48]:
ys

array([0, 3, 2, 1, 4])

In [49]:
np.unique(xs)

array([0, 2, 3, 4, 5])

In [50]:
def precision_recall(ious, gt_classes, pred_classes):
    """
    calculate precision and recall
    args:
    - ious [array]: NxM array of ious
    - gt_classes [array]: 1xN array of ground truth classes
    - pred_classes [array]: 1xM array of pred classes
    returns:
    - precision [float]
    - recall [float]
    """
    xs, ys = np.where(ious>0.5)

    # calculate true positive and true negative
    tps = 0
    fps = 0
    
    for x, y in zip(xs, ys):
        if gt_classes[x] == pred_classes[y]:
            tps += 1
        else:
            fps += 1

    matched_gt = len(np.unique(xs))
    fns = len(gt_classes) - matched_gt

    precision = tps / (tps+fps)
    recall = tps / (tps + fns)
    return precision, recall

In [51]:
ground_truth, predictions = get_data()
    
# get bboxes array
filename = 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'
gt_bboxes = [g['boxes'] for g in ground_truth if g['filename'] == filename][0]
gt_bboxes = np.array(gt_bboxes)

gt_classes = [g['classes'] for g in ground_truth if g['filename'] == filename][0]

pred_bboxes = [p['boxes'] for p in predictions if p['filename'] == filename][0]
pred_boxes = np.array(pred_bboxes)

pred_classes = [p['classes'] for p in predictions if p['filename'] == filename][0]

In [52]:
ious = calculate_ious(gt_bboxes, pred_boxes)
precision, recall = precision_recall(ious, gt_classes, pred_classes)

In [56]:
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Precision: 0.8
Recall: 0.5714285714285714
